# A Toy Viterbi Tagger (First Order Markov Assumption)

在这个练习里面，我需要用 Viterbi 算法来实现一个 toy POS tagger.  
在这个 toy tagger 里面：  

- tag 的类型一共只有 3 种： `NN` , `VV` 和 `IN`, 分别表示的是名词、动词和介词   
- tagger 使用`first order markov assumption`, 这意味着 `transistion probability` 的形式为 `p(s|u)`, 而不是 `Micheal` 课程里面的 `p(s|u,v)`   
- `transistion probabilities` 和 `emission probabilities` 都是已知的  


## Test Case 

- `mary lives in chengdu` 中的每一个词语分别应该被 Tag 成为： `NN VV IN NN`  

这里列出所有的 `transistion probability` 和 `emission probability`。  

在我们的 test case 里面，只有 3 种 tags (如果算上 `STOP` 就是 4 种)， 于是 `transistion probabilities` $q(s|u)$ 有以下这些情况：   


In [6]:
trans_p=dict();

# 句子开始的第一个 Tag 为 NN，VV或者IN 的概率 
trans_p['NN|*'] = 0.5; 
trans_p['VV|*'] = 0.1;
trans_p['IN|*'] = 0.4; 

# 然后是 9 种： p(s|u)  , s 和 u 各有 3 种可能 
trans_p['NN|NN'] = 0.2;
trans_p['VV|NN'] = 0.5;
trans_p['IN|NN'] = 0.3;

trans_p['NN|IN'] = 0.9;
trans_p['VV|IN'] = 0.05;
trans_p['IN|IN'] = 0.05;

trans_p['NN|VV'] = 0.5;
trans_p['VV|VV'] = 0.05;
trans_p['IN|VV'] = 0.45;

# 句子最后一个单词的 tag 
trans_p['STOP|NN'] = 0.8;
trans_p['STOP|VV'] = 0.15;
trans_p['STOP|IN'] = 0.05;


然后是 `emission probabilities` $e(x|s)$， 我们只考虑上面句子里的 4 个单词，然后这里有 4 种 tag ，于是 :  

In [7]:
emiss_p= dict();
 
emiss_p['mary|NN'] =  0.45
emiss_p['lives|NN']  =  0.05
emiss_p['in|NN'] = 0.05 
emiss_p['chengdu|NN'] =  0.45


emiss_p['mary|IN'] =  0.05
emiss_p['lives|IN']  =  0.05
emiss_p['in|IN'] = 0.85 
emiss_p['chengdu|IN'] =  0.05


emiss_p['mary|VV'] =  0.05
emiss_p['lives|VV']  =  0.85
emiss_p['in|VV'] = 0.05 
emiss_p['chengdu|VV'] =  0.05




那么，接下来的练习里面，我们应该能做出一个 tagger， 能够准确地 tag 出 `mary lives in chengdu` 这句话里的 `POS`.  

## Review of the generative model 

先来回忆一下这里的 `generative model`:  

$ p(x_1...x_n,y_1...y_{n+1}) = \prod_{i=1}^{n+1}q(y_i|y_{i-1})) \prod_{i=1}^{n}e(x_i|y_i) $   

其中 $y_n$ 是 `tag`, 而 $x_n$ 是 `word`.  
另外要注意的是 $y_{n+1} = STOP$ 

我们已经知道的 $x_1...x_n$, 需要找到一个 $y_1...y_{n+1}$ 让 $ p(x_1...x_n,y_1...y_{n+1})$ 的值是最大的。 

## Brute force way 

因为我们这里的句子很短， tags 的种类也很少，我们可以先试着用 `brute force way` 来看看。  
我们可以列举出所有的可能： 一共4个单词，每一个单词的 tag 有 3 种可能，于是一共是 81 种可能。  

In [11]:
all_possible_tagged_seqs = dict()

all_possible_tags = ['VV','NN','IN']

# FIXME: any other better way than 4 loops ?

for tag_of_mary in all_possible_tags:
    for tag_of_lives in all_possible_tags:
        for tag_of_in in all_possible_tags:
            for tag_of_chengdu in all_possible_tags:
                all_possible_tagged_seqs[tag_of_mary+' '+ tag_of_lives + ' ' + tag_of_in + ' ' + tag_of_chengdu] = (
                    trans_p[tag_of_mary+'|*'] * trans_p[tag_of_lives+'|' + tag_of_mary] *
                    trans_p[tag_of_in+'|'+tag_of_lives] * trans_p[tag_of_chengdu+'|'+tag_of_in] *  
                    trans_p['STOP|'+tag_of_chengdu] 
                    * emiss_p['mary|'+tag_of_mary] * emiss_p['lives|'+tag_of_lives] * 
                    emiss_p['in|'+tag_of_in] * emiss_p['chengdu|'+tag_of_chengdu] )

tagged_seq_with_max_prob = ''
max_prob = 0

for k,v in all_possible_tagged_seqs.items():
    if v > max_prob:
        max_prob = v 
        tagged_seq_with_max_prob = k
        
print('Tag Sequence of "'+ tagged_seq_with_max_prob + '" has the maximum probability of ' + str(max_prob))

Tag Sequence of "NN VV IN NN" has the maximum probability of 0.011850806250000002


## The Viterbi Way 




In [13]:
nodes = dict() 

# max_prob_to_this_node : 到这个 node 的各种 path 中， 概率最大值 
# pre_node : 到这个 node 的各种 path 中，概率最大的那个path 里，上一个 node 是什么 
nodes['*'] = {'max_prob_to_this_node':1, 'pre_node':'NOTHING'}

# 下面来考察和 'mary' 相关的 nodes
nodes['mary,NN'] = { 'max_prob_to_this_node': nodes['*']['max_prob_to_this_node'] 
                    * trans_p['NN|*'] * emiss_p['mary|NN'], 
                    'pre_node': '*'}

nodes['mary,VV'] = { 'max_prob_to_this_node': nodes['*']['max_prob_to_this_node'] 
                    * trans_p['VV|*'] * emiss_p['mary|VV'], 
                    'pre_node': '*'}

nodes['mary,IN'] = { 'max_prob_to_this_node': nodes['*']['max_prob_to_this_node'] 
                    * trans_p['IN|*'] * emiss_p['mary|IN'], 
                    'pre_node': '*'}

